In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.metrics import confusion_matrix
import time
from datetime import timedelta
import math


In [2]:
tf.__version__

'1.4.1'

In [3]:
# run Image_recognition module

%run 'Image_recognition.ipynb'

/usr/local/lib/python3.4/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


(39209, 2352) (12630, 2352)
(39209, 43) (12630, 43)


In [4]:
#After running Image_recognition.ipynb File ,you 
#can now access train test images and labels
#print all train test images and labels
# image size

print(train_images.shape,test_images.shape)
print(train_labels.shape,test_labels.shape)

(39209, 2352) (12630, 2352)
(39209, 43) (12630, 43)


In [5]:
# Conv Layer 1.
filter_size1 = 5        
num_filters1 = 16        

# Conv 2.
filter_size2 = 5          
num_filters2 = 36         

# Fully-connected layer.
fc_size = 128             

In [6]:
#Get all test labels (one hot Encoding to int labels) 
test_true_classes = np.argmax(test_labels, axis=1)

In [7]:
#weights function
def new_weights(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.05))

In [8]:
#Bias Function
def new_biases(length):
    return tf.Variable(tf.constant(0.05, shape=[length]))

In [9]:
def new_conv_layer(input, num_input_channels,filter_size,num_filters, use_pooling=True): 
    
    ''' create a new conv layer given parameter'''
    
    # defined Tensorflow Api
    shape = [filter_size, filter_size, num_input_channels, num_filters]

    #create weights
    weights = new_weights(shape=shape)

    #create biases
    biases = new_biases(length=num_filters)

    #create layer
    layer = tf.nn.conv2d(input=input,
                         filter=weights,
                         strides=[1, 1, 1, 1],
                         padding='SAME')
    #add biases
    layer += biases

    #use pooling if desired
    if use_pooling:

        layer = tf.nn.max_pool(value=layer,
                               ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='SAME')
    #use relu
    layer = tf.nn.relu(layer)

    return layer, weights

In [10]:
def flatten_layer(layer):
    
    ''' 
    flat all images in one single array
    so that ,we can use fully connected layer
    '''
    
    layer_shape = layer.get_shape()
    num_features = layer_shape[1:4].num_elements()
    layer_flat = tf.reshape(layer, [-1, num_features])
    return layer_flat, num_features

In [11]:
def new_fc_layer(input,num_inputs,num_outputs,use_relu=True): 

    ''' create a fully connected layer '''
    
    weights = new_weights(shape=[num_inputs, num_outputs])
    biases = new_biases(length=num_outputs)

    layer = tf.matmul(input, weights) + biases

    # use relu if desired
    if use_relu:
        layer = tf.nn.relu(layer)

    return layer

In [12]:
x = tf.placeholder(tf.float32, shape=[None, img_size_flat], name='x')

In [13]:
x_image = tf.reshape(x, [-1, img_size, img_size, num_channels])

In [14]:

y_true = tf.placeholder(tf.float32, shape=[None, num_classes], name='y_true')

In [15]:
y_true_cls = tf.argmax(y_true, axis=1)

In [16]:
#create first conv layer
layer_conv1, weights_conv1 = \
    new_conv_layer(input=x_image,
                   num_input_channels=num_channels,
                   filter_size=filter_size1,
                   num_filters=num_filters1,
                   use_pooling=True)

In [17]:
#show shape
layer_conv1

<tf.Tensor 'Relu:0' shape=(?, 14, 14, 16) dtype=float32>

In [18]:
#create second conv layer
layer_conv2, weights_conv2 = \
    new_conv_layer(input=layer_conv1,
                   num_input_channels=num_filters1,
                   filter_size=filter_size2,
                   num_filters=num_filters2,
                   use_pooling=True)

In [19]:
#show shape
layer_conv2

<tf.Tensor 'Relu_1:0' shape=(?, 7, 7, 36) dtype=float32>

In [20]:
#flatten layer
layer_flat, num_features = flatten_layer(layer_conv2)

In [21]:
#show shape
layer_flat

<tf.Tensor 'Reshape_1:0' shape=(?, 1764) dtype=float32>

In [22]:
#number of neuron from flatten layer
num_features

1764

In [23]:
#first fully connected layer
layer_fc1 = new_fc_layer(input=layer_flat,
                         num_inputs=num_features,
                         num_outputs=fc_size,
                         use_relu=True)

In [24]:
# ouput first fully layer
layer_fc1

<tf.Tensor 'Relu_2:0' shape=(?, 128) dtype=float32>

In [25]:
#create second fully connected layer
layer_fc2 = new_fc_layer(input=layer_fc1,
                         num_inputs=fc_size,
                         num_outputs=num_classes,
                         use_relu=False)

In [26]:
#output second fully connceted layer
layer_fc2

<tf.Tensor 'add_3:0' shape=(?, 43) dtype=float32>

In [29]:

y_pred = tf.nn.softmax(layer_fc2)

y_pred_cls = tf.argmax(y_pred, axis=1)

cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=layer_fc2,
                                                        labels=y_true)

cost = tf.reduce_mean(cross_entropy)

optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)

correct_prediction = tf.equal(y_pred_cls, y_true_cls)

accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

#create tensor session
session = tf.Session()

#initilize all variables
session.run(tf.global_variables_initializer())

In [30]:
train_batch_size = 64

In [31]:
def next_batch(num, data, labels):
    
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = data[idx]
    labels_shuffle = labels[idx]
    
    return data_shuffle, labels_shuffle

In [32]:

total_iterations = 0

def optimize(num_iterations):
    
    '''
    train network given num_iterations
    '''
    
    global total_iterations

    start_time = time.time()

    for i in range(total_iterations,
                   total_iterations + num_iterations):

        x_batch, y_true_batch = next_batch(train_batch_size, train_images, train_labels)

        feed_dict_train = {x: x_batch,
                           y_true: y_true_batch}

        session.run(optimizer, feed_dict=feed_dict_train)

        if i % 100 == 0:
            acc = session.run(accuracy, feed_dict=feed_dict_train)

            msg = "Optimization Iteration: {0:>6}, Training Accuracy: {1:>6.1%}"

            print(msg.format(i , acc))

    total_iterations += num_iterations

    end_time = time.time()

    time_dif = end_time - start_time

    print("Time usage: " + str(timedelta(seconds=int(round(time_dif)))))

In [33]:

test_batch_size = 256

def print_test_accuracy():

    num_test = len(test_images)
    cls_pred = np.zeros(shape=num_test, dtype=np.int)

    i = 0

    while i < num_test:
        j = min(i + test_batch_size, num_test)

        images = test_images[i:j, :]
        labels = test_labels[i:j, :]
        feed_dict = {x: images,
                     y_true: labels}
        cls_pred[i:j] = session.run(y_pred_cls, feed_dict=feed_dict)

        i = j

    cls_true = test_true_classes

    correct = (cls_true == cls_pred)

    correct_sum = correct.sum()

    acc = float(correct_sum) / num_test

    # Print the accuracy.
    msg = "Accuracy on Test-Set: {0:.1%} ({1} / {2})"
    print(msg.format(acc, correct_sum, num_test))


In [34]:
#test accuracy before traning
print_test_accuracy()

Accuracy on Test-Set: 4.4% (559 / 12630)


In [35]:
# train network
optimize(num_iterations=2000)

Optimization Iteration:      0, Training Accuracy:   9.4%
Optimization Iteration:    100, Training Accuracy:   1.6%
Optimization Iteration:    200, Training Accuracy:  15.6%
Optimization Iteration:    300, Training Accuracy:  15.6%
Optimization Iteration:    400, Training Accuracy:  18.8%
Optimization Iteration:    500, Training Accuracy:  21.9%
Optimization Iteration:    600, Training Accuracy:  31.2%
Optimization Iteration:    700, Training Accuracy:  35.9%
Optimization Iteration:    800, Training Accuracy:  34.4%
Optimization Iteration:    900, Training Accuracy:  34.4%
Optimization Iteration:   1000, Training Accuracy:  45.3%
Optimization Iteration:   1100, Training Accuracy:  48.4%
Optimization Iteration:   1200, Training Accuracy:  48.4%
Optimization Iteration:   1300, Training Accuracy:  51.6%
Optimization Iteration:   1400, Training Accuracy:  64.1%
Optimization Iteration:   1500, Training Accuracy:  57.8%
Optimization Iteration:   1600, Training Accuracy:  62.5%
Optimization I

In [36]:
# test accuracy after training
print_test_accuracy()

Accuracy on Test-Set: 60.4% (7626 / 12630)


In [50]:
# Close the session. This will destroy the trained model.
session.close()